In [19]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, row_number, lead, lag, count, avg

spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("apply-window-functions")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # Spark 설정 옵션
         .getOrCreate()  # SparkSession 생성 또는 기존 세션 반환)

spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [20]:
df = (spark.read
      .format("csv")
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("nullValue", "null")
      .option("dateFormat",  # 날짜 형식 지정 "LLLL d, y")
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [21]:
df = df.filter(  # 데이터 필터링col('country').isNotNull() & col('date_added').isNotNull())

In [22]:
from pyspark.sql.window import Window  # 윈도우 함수를 위한 Window 클래스 임포트
window_spec = Window.partitionBy("country").orderBy(  # 정렬"date_added")

In [23]:
# Assign row numbers within each partition
result = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"row_number", row_number().over(window_spec))
result.select("title","country","date_added","row_number").show()  # DataFrame 내용 출력

+--------------------+--------------------+--------------------+----------+
|               title|             country|          date_added|row_number|
+--------------------+--------------------+--------------------+----------+
| Beasts of No Nation|     Ama K. Abebrese|  Kobina Amissah Sam|         1|
|Get Him to the Greek|         Aziz Ansari|         Carla Gallo|         1|
|      Rhyme & Reason|            Chuck D.|     Desiree Densiti|         1|
|            Backfire|       Dominic Costa|        Nick Ferraro|         1|
|Hurricane Bianca:...|          Doug Plaut|    Cheyenne Jackson|         1|
|Offering to the S...|     Francesc Orella|        Imanol Arias|         1|
|        An Easy Girl|  Henri-Noël Tabary"|              France|         1|
| An Imperfect Murder|       James Toback"|       United States|         1|
|    Hurricane Bianca| Justin ""Alyssa ...|         Molly Ryman|         1|
|             Dayveon|  Lachion Buckingham|       Chasity Moore|         1|
|The Legacy 

In [24]:
# Add lead column
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"lead_date_added", lead("date_added").over(window_spec))
# Add lag column
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"lag_date_added", lag("date_added").over(window_spec))

df.select(  # 컬럼 선택"title","country","date_added","lead_date_added","lag_date_added").show(3)

+--------------------+----------------+-------------------+---------------+--------------+
|               title|         country|         date_added|lead_date_added|lag_date_added|
+--------------------+----------------+-------------------+---------------+--------------+
| Beasts of No Nation| Ama K. Abebrese| Kobina Amissah Sam|           null|          null|
|Get Him to the Greek|     Aziz Ansari|        Carla Gallo|           null|          null|
|      Rhyme & Reason|        Chuck D.|    Desiree Densiti|           null|          null|
+--------------------+----------------+-------------------+---------------+--------------+
only showing top 3 rows



### Nested Window Functions

In [25]:
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 sum, lead
from pyspark.sql.window import Window  # 윈도우 함수를 위한 Window 클래스 임포트

window_spec = Window.partitionBy("country").orderBy(  # 정렬"release_year")
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"running_total", count("show_id").over(window_spec))
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"next_running_total", lead("running_total").over(window_spec))
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"diff", df["next_running_total"] - df["running_total"])

### Window Frames

In [26]:
data = [(1, 10), (2, 15), (3, 20), (4, 25), (5, 30)]
df = spark.createDataFrame(data, ["id", "value"])

windowSpec = Window.orderBy(  # 정렬"id").rowsBetween(-2, 0)
df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"rolling_avg", avg(df["value"]).over(windowSpec))

df.show()  # DataFrame 내용 출력

+---+-----+-----------+
| id|value|rolling_avg|
+---+-----+-----------+
|  1|   10|       10.0|
|  2|   15|       12.5|
|  3|   20|       15.0|
|  4|   25|       20.0|
|  5|   30|       25.0|
+---+-----+-----------+



In [27]:
spark.stop()  # Spark 세션 종료 - 리소스 정리